In [2]:
import torch
import numpy as np

자동 미분

In [3]:
x = torch.ones(2, 2, requires_grad=True)

print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [4]:
z = 3*(x+2)**2

print(z)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)


In [5]:
# x.backward() == x.backward(torch.tensor(1))
z.backward(torch.tensor([[1, 1], [1, 0.1]]))

print(x.grad)

tensor([[18.0000, 18.0000],
        [18.0000,  1.8000]])


In [6]:
print(x.norm())

tensor(2., grad_fn=<NormBackward0>)


In [7]:
with torch.no_grad():
    # don't track
    y1 = x+3
y2 = x**2

In [8]:
print(y1.requires_grad)
print(y2.requires_grad)

False
True


In [9]:
x1 = x.detach()

print(x1.requires_grad)
print(x1.eq(x))

False
tensor([[True, True],
        [True, True]])


### Define new autograd function

* static method로 forward와 backward함수를 구현해야 함
* `forward(ctx, tensor_i) -> tensor_o`
    * ctx(context object) backward에서 연산을 위해 필요한 값을 저장하는 변수
        * ctx.save_for_backward(tensor)
    * 연산된 tensor를 반환
* `backward(ctx, grad_o) -> grad_i`
    * 출력의 손실 변화도와 저장된 값을 이용해 입력 텐서의 손실 변화도를 계산해 반환

In [10]:
class MyReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, tensor_i):
        ctx.save_for_backward(tensor_i)
        return tensor_i.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_o):
        tensor_i, = ctx.saved_tensors
        grad_i = grad_o.clone()
        grad_i[tensor_i < 0] = 0
        return grad_i